<a href="https://colab.research.google.com/github/Jun1233211/CF-Workers-SUB/blob/main/%E6%89%B9%E9%87%8F%E4%B8%8B%E8%BD%BD%E6%A8%A1%E5%9E%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# huggingface下载


In [31]:
import os
from google.colab import drive
from huggingface_hub import hf_hub_download # 添加用于 Hugging Face 下载

# --- Google Drive 基路径 ---
# 所有模型将下载到此路径下的子文件夹中。
GOOGLE_DRIVE_BASE_PATH = "/content/drive/MyDrive/comfy_model/" #@param {type:"string"}

# --- 挂载 Google Drive ---
print("正在挂载 Google Drive...")
drive.mount('/content/drive')
print("Google Drive 已挂载。")

# 安装 aria2c (只安装一次)
print("正在安装 aria2c...")
!apt-get -qq install -y aria2
print("aria2c 已安装。")

# --- 模型配置块 1 ---
# 请根据您要下载的模型填写以下参数。
# 如果 MODEL_URL 非空，则优先使用 MODEL_URL 进行下载。
MODEL_URL_1 = "https://huggingface.co/comfyanonymous/flux_text_encoders/resolve/main/t5xxl_fp16.safetensors?download=true" #@param {type:"string"}
MODEL_TYPE_1 = "text_encoders" #@param ["text_encoders", "clip_vision", "loras", "unet", "other"] {type:"string"}
FILE_NAME_1 = "t5xxl_fp16.safetensors" #@param {type:"string"}
REPO_ID_1 = "" #@param {type:"string"}
HF_FILE_NAME_1 = "" #@param {type:"string"}


# --- 模型配置块 2 ---
MODEL_URL_2 = "" #@param {type:"string"}
MODEL_TYPE_2 = "unet" #@param ["text_encoders", "clip_vision", "loras", "unet", "other"] {type:"string"}
FILE_NAME_2 = "hidream-i1-fast-Q4_K_M.gguf" #@param {type:"string"}
REPO_ID_2 = "city96/HiDream-I1-Fast-gguf" #@param {type:"string"}
HF_FILE_NAME_2 = "hidream-i1-fast-Q4_K_M.gguf" #@param {type:"string"}


# --- 模型配置块 3 ---
# 您可以复制并粘贴以上块来添加更多模型。
MODEL_URL_3 = "https://huggingface.co/ratoenien/clip_vision_h/resolve/main/clip_vision_h.safetensors?download=true" #@param {type:"string"}
MODEL_TYPE_3 = "clip_vision" #@param ["text_encoders", "clip_vision", "loras", "unet", "other"] {type:"string"}
FILE_NAME_3 = "clip_vision_h.safetensors" #@param {type:"string"}
REPO_ID_3 = "" #@param {type:"string"}
HF_FILE_NAME_3 = "" #@param {type:"string"}


# 收集所有模型参数到一个列表中
# 您需要手动在此处添加您创建的每个模型配置块的变量。
# 如果某个块的参数没有填写（例如URL和Repo ID都为空），它将被跳过。
all_models_params = [
    {"url": MODEL_URL_1, "type": MODEL_TYPE_1, "file_name": FILE_NAME_1, "repo_id": REPO_ID_1, "hf_file_name": HF_FILE_NAME_1},
    {"url": MODEL_URL_2, "type": MODEL_TYPE_2, "file_name": FILE_NAME_2, "repo_id": REPO_ID_2, "hf_file_name": HF_FILE_NAME_2},
    {"url": MODEL_URL_3, "type": MODEL_TYPE_3, "file_name": FILE_NAME_3, "repo_id": REPO_ID_3, "hf_file_name": HF_FILE_NAME_3},
    # 复制并粘贴此行以添加更多模型块的参数
    # {"url": MODEL_URL_4, "type": MODEL_TYPE_4, "file_name": FILE_NAME_4, "repo_id": REPO_ID_4, "hf_file_name": HF_FILE_NAME_4},
]

# --- 依次下载模型 ---
for i, model_params in enumerate(all_models_params):
    current_model_url = model_params["url"]
    current_model_type = model_params["type"]
    current_file_name = model_params["file_name"]
    current_repo_id = model_params["repo_id"]
    current_hf_file_name = model_params["hf_file_name"]

    # 如果所有关键参数都为空，则跳过此模型
    if not current_model_url and (not current_repo_id or not current_hf_file_name):
        print(f"\n--- 模型配置块 {i+1}: 参数为空，跳过此模型。---")
        continue

    print(f"\n--- 正在处理模型 {i+1}: {current_file_name} ---")

    # 根据 MODEL_TYPE 构建目标路径
    drive_models_path = os.path.join(GOOGLE_DRIVE_BASE_PATH, current_model_type)
    os.makedirs(drive_models_path, exist_ok=True)
    destination_path = os.path.join(drive_models_path, current_file_name)

    if os.path.exists(destination_path):
        print(f"✅ 文件 '{current_file_name}' 已经存在于 '{destination_path}'，跳过下载。")
    else:
        if current_model_url:
            # 使用 aria2c 进行直接 URL 下载
            print(f"🚀 正在使用 aria2c 下载: {current_model_url}")
            # 下载到 /content 临时目录
            !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{current_model_url}" -d /content -o {current_file_name}
            source_path = os.path.join("/content", current_file_name)

            # 检查文件是否下载成功
            if os.path.exists(source_path) and os.path.getsize(source_path) > 0:
                print(f"🚚 正在移动到: {destination_path}")
                !mv "{source_path}" "{destination_path}"
                print(f"✅ 已将 '{current_file_name}' 下载并移动到 {destination_path}")
            else:
                print(f"❌ 下载失败或文件为空：{current_file_name}")

        elif current_repo_id and current_hf_file_name:
            # 使用 huggingface_hub 进行仓库下载
            print(f"🚀 正在使用 huggingface_hub 下载：从仓库 '{current_repo_id}' 下载文件 '{current_hf_file_name}'")
            try:
                hf_hub_download(
                    repo_id=current_repo_id,
                    filename=current_hf_file_name,
                    cache_dir=drive_models_path, # 直接下载到最终目的地
                    force_download=False, # 如果已在缓存目录中，则不再重新下载
                    local_dir_use_symlinks=False # 避免符号链接，这可能与 Colab/Drive 存在问题
                )
                # 由于 HF_FILE_NAME 可能与 FILE_NAME 不同，需要确保最终路径正确
                downloaded_hf_path = os.path.join(drive_models_path, current_hf_file_name)
                if current_hf_file_name != current_file_name and os.path.exists(downloaded_hf_path):
                    final_destination_path = os.path.join(drive_models_path, current_file_name)
                    print(f"🚚 正在重命名 '{current_hf_file_name}' 为 '{current_file_name}' 并移动到: {final_destination_path}")
                    !mv "{downloaded_hf_path}" "{final_destination_path}"
                    print(f"✅ 已将 '{current_file_name}' 下载并保存到 {final_destination_path}")
                else:
                    print(f"✅ 已将 '{current_file_name}' 下载并保存到 {destination_path}")

            except Exception as e:
                print(f"❌ 从 {current_repo_id} 下载 {current_hf_file_name} 失败: {e}")
        else:
            print("⚠️ 未指定有效的下载源（URL 或 Hugging Face 仓库信息）。请填写模型参数。")

print("\n🎉 所有模型处理完成。以下是各文件夹内容：\n")

# --- 列出所有模型类型文件夹的内容 ---
# 收集所有独特的模型类型
unique_model_types = sorted(list(set(m["type"] for m in all_models_params if m and m.get("type"))))

for model_type in unique_model_types:
    drive_models_path = os.path.join(GOOGLE_DRIVE_BASE_PATH, model_type)
    if os.path.exists(drive_models_path):
        print(f"\n📂 {model_type} 文件夹:")
        !ls -lh "{drive_models_path}"
    else:
        print(f"\n📂 {model_type} 文件夹: 未找到或为空。")

正在挂载 Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive 已挂载。
正在安装 aria2c...
aria2c 已安装。

--- 正在处理模型 1: t5xxl_fp16.safetensors ---
✅ 文件 't5xxl_fp16.safetensors' 已经存在于 '/content/drive/MyDrive/comfy_model/text_encoders/t5xxl_fp16.safetensors'，跳过下载。

--- 正在处理模型 2: hidream-i1-fast-Q4_K_M.gguf ---
🚀 正在使用 huggingface_hub 下载：从仓库 'city96/HiDream-I1-Fast-gguf' 下载文件 'hidream-i1-fast-Q4_K_M.gguf'


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


hidream-i1-fast-Q4_K_M.gguf:   0%|          | 0.00/11.5G [00:00<?, ?B/s]

✅ 已将 'hidream-i1-fast-Q4_K_M.gguf' 下载并保存到 /content/drive/MyDrive/comfy_model/unet/hidream-i1-fast-Q4_K_M.gguf

--- 正在处理模型 3: clip_vision_h.safetensors ---
✅ 文件 'clip_vision_h.safetensors' 已经存在于 '/content/drive/MyDrive/comfy_model/clip_vision/clip_vision_h.safetensors'，跳过下载。

🎉 所有模型处理完成。以下是各文件夹内容：


📂 clip_vision 文件夹:
total 1.2G
-rw-------+ 1 root root 1.2G Jul 15 05:14 clip_vision_h.safetensors

📂 text_encoders 文件夹:
total 42G
-rw-------+ 1 root root 235M Jul 15 05:08 clip_l.safetensors
-rw-------+ 1 root root 9.2G Jul 15 05:09 t5xxl_fp16.safetensors
-rw-------+ 1 root root 4.6G Jul 15 05:11 t5xxl_fp8_e4m3fn.safetensors
-rw-------+ 1 root root 4.9G Jul 15 05:13 t5xxl_fp8_e4m3fn_scaled.safetensors
-rw-------  1 root root 5.7G Jul 14 11:49 umt5-xxl-encoder-Q8_0.gguf
-rw-------  1 root root  11G Jul 14 11:35 umt5_xxl_fp16.safetensors
-rw-------  1 root root 6.3G Jul 14 11:32 umt5_xxl_fp8_e4m3fn_scaled.safetensors

📂 unet 文件夹:
total 12G
-rw-------+ 1 root root  12G Jul 15 06:31 flux1-fill-d

# C站下载

In [33]:
import os
from google.colab import drive
import requests # 用于 HTTP 请求

# --- 用户配置：Civitai API Key (可选) ---
# 如果您需要下载私人模型或遇到速率限制，请在此处填写您的 Civitai API Key。
# 您可以在 Civitai 网站的个人设置中生成它：https://civitai.com/user/account
# 如果您只下载公开模型，可以留空。
CIVITAI_API_KEY = "49af372753015ed6a504664c16061ec7"

# --- Google Drive 基路径 ---
# 所有模型将下载到此路径下的子文件夹中。
GOOGLE_DRIVE_BASE_PATH = "/content/drive/MyDrive/comfy_model/"

# --- 挂载 Google Drive ---
print("正在挂载 Google Drive...")
drive.mount('/content/drive')
print("Google Drive 已挂载。")

# 安装 aria2c (只安装一次)
print("正在安装 aria2c...")
!apt-get -qq install -y aria2
print("aria2c 已安装。")

# --- 辅助函数：带授权的下载 ---
def download_file_with_auth(url, destination_path, file_name, api_key=None):
    headers = {}
    if api_key:
        headers["Authorization"] = f"Bearer {api_key}"
        print("尝试使用 API Key 进行下载...")

    try:
        # 使用 requests 流式下载，以便支持认证头
        with requests.get(url, headers=headers, stream=True, allow_redirects=True) as r:
            r.raise_for_status() # 检查 HTTP 错误

            # 如果响应指示下载失败或需要认证，考虑使用 aria2c fallback
            # 某些Civitai链接可能仍然是重定向到CDN，requests能处理，但如果遇到特定认证问题，
            # 可能需要更复杂的重试逻辑。这里暂时简化。

            file_size = int(r.headers.get('content-length', 0))
            downloaded_size = 0

            with open(destination_path, 'wb') as f:
                for chunk in r.iter_content(chunk_size=8192):
                    if chunk: # filter out keep-alive new chunks
                        f.write(chunk)
                        downloaded_size += len(chunk)
                        # 可以在这里添加进度条逻辑，例如 tqdm
                        # print(f"\r下载进度: {downloaded_size / (1024 * 1024):.2f}MB / {file_size / (1024 * 1024):.2f}MB", end="")
            print(f"\n✅ 已将 '{file_name}' 下载并保存到 {destination_path}")
            return True
    except requests.exceptions.RequestException as e:
        print(f"❌ 下载 {file_name} 失败 (Requests 错误): {e}")
        # 如果 requests 失败，尝试使用 aria2c 作为回退 (不带认证头)
        print("尝试使用 aria2c 作为回退下载...")
        # 注意：aria2c 不支持直接的 Bearer Token 认证头，仅用于公共URL
        try:
            !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{url}" -d {os.path.dirname(destination_path)} -o {file_name}
            if os.path.exists(destination_path) and os.path.getsize(destination_path) > 0:
                print(f"✅ (回退) 已将 '{file_name}' 下载并保存到 {destination_path}")
                return True
            else:
                print(f"❌ (回退) 下载失败或文件为空：{file_name}")
                return False
        except Exception as aria_e:
            print(f"❌ (回退) aria2c 下载也失败: {aria_e}")
            return False
    except Exception as e:
        print(f"❌ 下载 {file_name} 失败: {e}")
        return False

# --- 模型配置块 1 ---
# Civitai 模型下载通常直接提供模型文件的下载链接（位于模型详情页的"Download"按钮下方或版本信息中）。
# 请查找以 .safetensors, .ckpt, .bin, .pth, .pt, .onnx, .gguf 等结尾的直接下载链接。
MODEL_NAME_1 = "Medieval Manuscript Style" #@param {type:"string"}
MODEL_URL_1 = "https://civitai.com/api/download/models/1967449?type=Model&format=SafeTensor" #@param {type:"string"}
MODEL_TYPE_1 = "loras" #@param ["checkpoint", "text_encoders", "clip_vision", "loras", "embeddings", "vae", "controlnet", "other"] {type:"string"}
FILE_NAME_1 = "Medieva_Manuscript_Style.safetensors" #@param {type:"string"}


# --- 模型配置块 2 ---
MODEL_NAME_2 = "" #@param {type:"string"}
MODEL_URL_2 = "" #@param {type:"string"}
MODEL_TYPE_2 = "other" #@param ["checkpoint", "text_encoders", "clip_vision", "loras", "embeddings", "vae", "controlnet", "other"] {type:"string"}
FILE_NAME_2 = "another_civitai_model.safetensors" #@param {type:"string"}


# --- 模型配置块 3 ---
# 复制并粘贴以上块来添加更多模型。
MODEL_NAME_3 = "Civitai_Embedding" #@param {type:"string"}
MODEL_URL_3 = None #@param {type:"string"} # 注意类型和文件名
MODEL_TYPE_3 = "embeddings" #@param ["checkpoint", "text_encoders", "clip_vision", "loras", "embeddings", "vae", "controlnet", "other"] {type:"string"}
FILE_NAME_3 = "" #@param {type:"string"}


# 收集所有模型参数到一个列表中
# 您需要手动在此处添加您创建的每个模型配置块的变量。
# 如果某个块的参数没有填写（例如 URL 为空），它将被跳过。
all_civitai_models = [
    {"name": MODEL_NAME_1, "url": MODEL_URL_1, "type": MODEL_TYPE_1, "file_name": FILE_NAME_1},
    {"name": MODEL_NAME_2, "url": MODEL_URL_2, "type": MODEL_TYPE_2, "file_name": FILE_NAME_2},
    {"name": MODEL_NAME_3, "url": MODEL_URL_3, "type": MODEL_TYPE_3, "file_name": FILE_NAME_3},
    # 复制并粘贴此行以添加更多模型块的参数
    # {"name": MODEL_NAME_4, "url": MODEL_URL_4, "type": MODEL_TYPE_4, "file_name": FILE_NAME_4},
]

# --- 依次下载模型 ---
processed_model_types = set()

for i, model_data in enumerate(all_civitai_models):
    model_name = model_data.get("name", f"模型 {i+1}")
    model_url = model_data.get("url", "")
    model_type = model_data.get("type", "")
    file_name = model_data.get("file_name", "")

    # 检查基本参数是否完整
    if not model_url or not model_type or not file_name:
        print(f"\n--- 模型配置块 {i+1} ({model_name}): 参数不完整 (URL, 类型或文件名缺失)，跳过此模型。---")
        continue

    print(f"\n--- 正在处理模型 {i+1}: {model_name} ({file_name}) ---")
    processed_model_types.add(model_type) # 记录当前处理的模型类型

    # 根据 MODEL_TYPE 构建目标路径
    drive_models_path = os.path.join(GOOGLE_DRIVE_BASE_PATH, model_type)
    os.makedirs(drive_models_path, exist_ok=True)
    destination_path = os.path.join(drive_models_path, file_name)

    if os.path.exists(destination_path) and os.path.getsize(destination_path) > 0:
        print(f"✅ 文件 '{file_name}' 已经存在于 '{destination_path}'，跳过下载。")
    else:
        # 尝试使用 requests 和 API Key 下载
        print(f"🚀 尝试下载: {file_name} from {model_url}")
        download_success = download_file_with_auth(model_url, destination_path, file_name, CIVITAI_API_KEY)

        if not download_success:
            print(f"❌ 无法下载 '{file_name}'。请检查 URL、网络连接或 API Key 是否正确。")

print("\n🎉 所有模型处理完成。以下是各文件夹内容：\n")

# --- 列出所有模型类型文件夹的内容 ---
unique_model_types_to_list = sorted(list(processed_model_types))

for model_type in unique_model_types_to_list:
    drive_models_path = os.path.join(GOOGLE_DRIVE_BASE_PATH, model_type)
    if os.path.exists(drive_models_path):
        print(f"\n📂 {model_type} 文件夹:")
        !ls -lh "{drive_models_path}"
    else:
        print(f"\n📂 {model_type} 文件夹: 未找到或为空。")

正在挂载 Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive 已挂载。
正在安装 aria2c...
aria2c 已安装。

--- 正在处理模型 1: Medieval Manuscript Style (Medieva_Manuscript_Style.safetensors) ---
🚀 尝试下载: Medieva_Manuscript_Style.safetensors from https://civitai.com/api/download/models/1967449?type=Model&format=SafeTensor
尝试使用 API Key 进行下载...

✅ 已将 'Medieva_Manuscript_Style.safetensors' 下载并保存到 /content/drive/MyDrive/comfy_model/loras/Medieva_Manuscript_Style.safetensors

--- 模型配置块 2 (): 参数不完整 (URL, 类型或文件名缺失)，跳过此模型。---

--- 模型配置块 3 (Civitai_Embedding): 参数不完整 (URL, 类型或文件名缺失)，跳过此模型。---

🎉 所有模型处理完成。以下是各文件夹内容：


📂 loras 文件夹:
total 1.6G
-rw-------  1 root root 328M Jul 15 07:27 Medieva_Manuscript_Style.safetensors
-rw-------  1 root root 305M Jun 30 06:35 Wan21_8步CausVid_14B_T2V_lora.safetensors
-rw-------+ 1 root root 305M Jul 15 05:15 Wan21_CausVid_14B_T2V_lora_rank32.safetensors
-rw-------+ 1 root root 303M 